# Example formatted output in multiple formats for IA

- Show a data load
- Show a manipulation
- Show how a map loads/exports

Proposal:

A Jupyter Notebook (in the style of https://www.theatlantic.com/science/archive/2018/04/the-scientific-paper-is-obsolete/556676/ with text + code. A more emphatic version of https://mybinder.org/v2/gh/mqAncientHistory/Lat-Epig/HEAD?urlpath=notebooks/EpigraphyScraper.ipynb) with a normal (markdown formatted) journal article + integration with OpenAI's large language models to provide examples (and an interface for readers to put their own epigraphy through, so long as they have an API key).

This proposal is intended to start discussion of how these Large Language Models could be used (or require further development before re-assessing) in Latin epigraphic research. In many wise, it is intended to start a discussion of how these tools _should_ be used in research.

We plan to use our published dataset to assess the quality of different large language model prompts (along with experimenting with [Microsoft's Guidance library](https://github.com/microsoft/guidance) for more effective "prompt engineering."

We also plan to provoke/invite responses by other scholars around the application of the tool and future directions in the fields of Archaeology and Epigraphy


In [15]:

# load a JSON file

import json
from pprint import pprint

with open('2023-05-12-EDCS_via_Lat_Epig-term1_tumulus-74.json') as f:
    data = json.load(f)

# make some summary statistics about the number of inscriptions in the file
# data dict_keys(['metadata', 'date', 'data'])

print("Search Terms:")

for key, value in data['metadata'].items():
    if value:
        print(f"* {key}: {value}")


print(f"Data fetched on: {data['date']}")
print("Number of Inscriptions:", len(data['data']))

# find the inscriptions with the smallest and biggest longitude and latitude

min_lat = 1000
max_lat = -1000
min_lat_id = ""
min_long_id = ""
min_lon = 1000
max_lon = -1000
max_lon_id = ""
max_lat_id = ""

for inscription in data['data']:
    if not inscription['latitude']:
        continue
    if float(inscription['latitude']) < min_lat:
        min_lat = float(inscription['latitude'])
        min_lat_id = inscription['EDCS-ID']
    if float(inscription['latitude']) > max_lat:
        max_lat = float(inscription['latitude'])
        max_lat_id = inscription['EDCS-ID']
    if float(inscription['longitude']) < min_lon:
        min_lon = float(inscription['longitude'])
        min_long_id = inscription['EDCS-ID']
    if float(inscription['longitude']) > max_lon:
        max_lon = float(inscription['longitude'])
        max_lon_id = inscription['EDCS-ID']

print(f"Min Latitude: {min_lat} ({min_lat_id})")
print(f"Max Latitude: {max_lat} ({max_lat_id})")
print(f"Min Longitude: {min_lon} ({min_long_id})")
print(f"Max Longitude: {max_lon} ({max_lon_id})")


Search Terms:
* operator: and
* term1: tumulus
Data fetched on: 2023-05-12T14:26:28.191272
Number of Inscriptions: 74
Min Latitude: 35.554783 (EDCS-08800049)
Max Latitude: 48.8567811 (EDCS-55000516)
Min Longitude: -7.8105708 (EDCS-08500330)
Max Longitude: 35.1506765 (EDCS-24100170)


Above is an example of loading a file and making some notional summary statistics. Obviouly, this is abbreviated in this demo, but here is where we would be putting the "traditional academic" portions of the paper.

# How to use the paper

# Introduction

# Aims

... etc...

Now, we will show a basic manipulation (a dropdown menu by which one of the terms can be selected), what that looks like in an archived state, and a [slippymap](https://wiki.openstreetmap.org/wiki/Slippy_map) (and its archive).


In [21]:
# make a ipybwidget dropdown for the inscriptions. The dropdown should be f"{EDCS-ID}: truncated 50 characters of inscription_interpretive_cleaning

from ipywidgets import interact, Dropdown, widgets, HTML, Button
from IPython.display import display_markdown

# Convert data['data'] into a dict appropriate for make_dropdown. data['data'] is a list of dicts.

inscriptions = {}
for item in data['data']:
    # print(item.keys())
    inscriptions[item['EDCS-ID']] = {'edcs_id': item['EDCS-ID'], 
                                    'inscription_interpretive_cleaning': item['inscription_interpretive_cleaning'],
                                     'data': item
                                    }
# order the inscriptions by EDCS-ID
inscriptions = dict(sorted(inscriptions.items()))

def make_dropdown(x):
    """
    Return a dropdown widget with inscription IDs and truncated inscriptions. The value should be the EDCS-ID
    """
    dropdown = Dropdown(
        options=[(f"{inscription['edcs_id']}: {inscription['inscription_interpretive_cleaning'][:50]}...", inscription['edcs_id']) for inscription in inscriptions.values()],
        description='Inscriptions:',
        disabled=False,
        # make this dropdown 100% wide with wrapping
        layout={'width': '100%'}

    )
    return dropdown

dropdown = make_dropdown(inscriptions)
display(dropdown)

# make a output capture widget for the inscription on change


# out = widgets.Output(layout={'border': '0px solid black'})
# display(out)

# @out.capture()
def generate_output(change):
    """Makes the output for a given inscription choice.
    """
    display_markdown(f"## Inscription {inscriptions[change['new']]['data']['EDCS-ID']}", raw=True)    

    for key, value in inscriptions[change['new']]['data'].items():
        if key == "EDCS-ID":
            continue
        if value:
            display_markdown(f"* **{key}**: {value}", raw=True)
    



def inscription_dropdown_on_change(change):
    """
    # when they choose an item on the dropdown, print the original full inscription and its metadata.
    """

    if change['type'] == 'change' and change['name'] == 'value':
        # print("changed to %s" % change['new'])
        # print(inscriptions[change['new']]['data']['inscription_interpretive_cleaning'])
        # pprint(inscriptions[change['new']]['data'])
        # out.clear_output()
        generate_output(change)

        

dropdown.observe(inscription_dropdown_on_change)


# make a default selection, plus printed output of EDCS-09200587 so that when the notebook starts, that is selected with the output.

dropdown.value = 'EDCS-09200587'


Dropdown(description='Inscriptions:', layout=Layout(width='100%'), options=(('EDCS-00900052: placere uti ianus…

## Inscription EDCS-09200587

## Inscription EDCS-10200960

# Static Map

We will generate a map, in this example with the long, lat of the selected inscription as the centre, plus a standard view around it.


In [17]:

# load pygeos, geopandas, natural_earth

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from geodatasets import get_path

# make a geometry column from the lat/long data
from shapely import wkt

# make a button for render point
button = Button(description="Render Point")
display(button)



def render_point(edcs_id, lat, long, inscription):
    """Return a shapely point from the lat/long data in the selected_data dict.
    selected_data = inscriptions[dropdown.value]['data']
    """
    
    selected_data = inscription
    selected_data['geometry'] = wkt.loads(f"POINT({long} {lat})")
    selected_data['properties'] = {'ID':edcs_id}
    print(lat, long, edcs_id)
    # load the EDCS selected data into a geopandas dataframe

    df = gpd.GeoDataFrame.from_features([inscriptions[dropdown.value]['data']])
    df.crs = "EPSG:4326"

    # # load natural earth as the basemap
    # ax = df.plot()

    # write the plot to map_output

    display(HTML(f"<h2>Inscription {edcs_id}</h2>"))
    world = gpd.read_file(get_path("naturalearth.land"))

    # clip the world to the buffer
    ax = world.clip([-15,25,50,65]).plot(color="white", edgecolor="black")
    # plot the point
    df.plot(ax=ax, color="red")



def button_on_click(b):
    """When the button is clicked, render the point.
    """
    # print("clicked", dropdown.value, inscriptions[dropdown.value]['data']['latitude'], inscriptions[dropdown.value]['data']['longitude'])
    render_point(edcs_id=dropdown.value, lat=inscriptions[dropdown.value]['data']['latitude'], long=inscriptions[dropdown.value]['data']['longitude'], inscription=inscriptions[dropdown.value]['data'])

button.on_click(button_on_click)

Button(description='Render Point', style=ButtonStyle())